In [8]:
from flask import Flask, request, jsonify

In [9]:
app = Flask(__name__)
app.config['DEBUG'] = False

In [10]:

books = [ 
            {'id': 0, 
            'title': 'A Fire Upon the Deep', 
            'author': 'VernorVinge', 
            'first_sentence': 'The coldsleep itself was dreamless.', 
            'published': '1992'},
            
            {'id': 1, 
            'title': 'The Ones Who Walk Away From Omelas', 
            'author': 'Ursula K. Le Guin', 
            'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.', 
            'published': '1973'},

            {'id': 2, 
            'title': 'Dhalgren', 
            'author': 'Samuel R. Delany', 
            'first_sentence': 'to wound the autumnal city.', 
            'published': '1975'} 
        ]



In [11]:
import pandas as pd
import sqlite3
books = pd.read_csv("books.csv")
books["published"] = books["published"].astype(int)

In [12]:
@app.route('/', methods=['GET'])
def home():
	return "<h1>La Enciclopedia de Hari Seldon</h1><p>This site is a prototype API for distant reading of science fiction novels.</p>"


# GET Display all the books
#@app.route('/api/v1/resources/books/all', methods=['GET']) 
#def api_all():
#    return jsonify(books.to_dict(orient="records"))
    #return books
@app.route('/api/v1/resources/books/all', methods=['GET']) 
def get_all(): 
    connection = sqlite3.connect('data/books.db') 
    cursor = connection.cursor() 
 
    select_books = "SELECT * FROM books" 
 
    result = cursor.execute(select_books).fetchall() 
 
    connection.close() 
 
    return {'books': result} 

@app.route('/api/v1/resources/books/1', methods=['GET']) 
def api_1():
    return jsonify(books.iloc[0].to_dict())

@app.route('/api/v1/resources/books/2', methods=['GET']) 
def api_2():
    return jsonify(books.iloc[1].to_dict())

@app.route('/api/v1/resources/books/3', methods=['GET']) 
def api_3():
    return jsonify(books.iloc[2].to_dict())

@app.route('/api/v1/resources/books', methods=['GET']) 
def api_id(): 
 
    if 'id' in request.args: 
        id = int(request.args['id']) 
        results = books[books["id"]==id]
    elif 'author' in request.args: 
        author = request.args['author']
        results = books[books["author"]==author]
    else: 
        return "Error: No id field provided. Please specify an id." 
    
 
    return jsonify(results.to_dict())

@app.route('/api/v1/resources/query', methods=['GET']) 
def api_query():
    from_ = int(request.args["from"])
    to_ =  int(request.args["to"])
    #results = books[(books["published"]>=from_) & (books["published"]<=to_)]
    connection = sqlite3.connect('data/books.db') 
    cursor = connection.cursor() 
 
    select_books = "SELECT * FROM books WHERE published>=? AND published<=?" 
 
    result = cursor.execute(select_books, (from_, to_)).fetchall() 
 
    connection.close() 
 
    return {'books': result} 
    return jsonify(results.to_dict(orient="records"))

@app.route('/api/v1/resources/books/save', methods=['GET']) 
def api_save(): 
 
    books.to_csv("books.csv")
 
    return jsonify("Se ha guardado")




app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Jul/2024 12:33:48] "GET /api/v1/resources/books/all HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2024 14:14:03] "GET /api/v1/resources/books?id=1 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2024 14:14:29] "GET /api/v1/resources/books?id=1&author=Ursula%20K.%20Le%20Guin HTTP/1.1" 200 -
